# Store Sales - Time Series Forecasting
*Use machine learning to predict grocery sales*

Background:
- Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
- A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [365]:
# TODO
"""
- add holidays
- add transactions
- add other methods: XGB, Cat, NN
"""

'\n- add holidays\n- add transactions\n- add other methods: XGB, Cat, NN\n'

In [366]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid", palette="muted")

RANDOM_STATE = 101


### Data collection

In [367]:
df_train = pd.read_csv(
    "./data/train.csv",
    usecols=["id", "date", "store_nbr", "family", "sales", "onpromotion"],
    dtype={
        "store_nbr": "category",
        "family": "category",
        "sales": "float32",
        "onpromotion": "uint32",
    },
    parse_dates=["date"],
)

df_train.shape

(3000888, 6)

In [368]:
pd.concat([df_train.head(2), df_train.tail(2)])

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.0,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.0,0


In [369]:
df_test = pd.read_csv(
    "./data/test.csv",
    usecols=["id", "date", "store_nbr", "family", "onpromotion"],
    dtype={
        "store_nbr": "category",
        "family": "category",
        "onpromotion": "uint32",
    },
    parse_dates=["date"],
)

df_test.shape

(28512, 5)

In [370]:
pd.concat([df_test.head(2), df_test.tail(2)])

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9
28511,3029399,2017-08-31,9,SEAFOOD,0


In [371]:
df_holidays = pd.read_csv(
    "./data/holidays_events.csv",
    # dtype={
    #     "type": "category",
    #     "locale": "category",
    # },
    parse_dates=["date"],
)

df_holidays.shape

(350, 6)

In [372]:
pd.concat([df_holidays.head(2), df_holidays.tail(2)])

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False
349,2017-12-26,Additional,National,Ecuador,Navidad+1,False


In [373]:
df_stores = pd.read_csv(
    "./data/stores.csv",
    dtype={
        "store_nbr": "category",
        "city": "category",
        "state": "category",
        "type": "category",
        "cluster": "category",
    },
)

df_holidays.shape

(350, 6)

In [374]:
pd.concat([df_stores.head(2), df_stores.tail(2)])

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
52,53,Manta,Manabi,D,13
53,54,El Carmen,Manabi,C,3


In [375]:
df_oil = pd.read_csv(
    "./data/oil.csv",
    parse_dates=["date"],
    # index_col="date",
)

df_oil.shape

(1218, 2)

In [376]:
pd.concat([df_oil.head(2), df_oil.tail(2)])

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
1216,2017-08-30,45.96
1217,2017-08-31,47.26


In [377]:
df_txns = pd.read_csv(
    "./data/transactions.csv",
    dtype={
        "store_nbr": "category",
        "transactions": "uint32",
    },
    parse_dates=["date"],
    # index_col="date",
)

df_txns.shape

(83488, 3)

In [378]:
pd.concat([df_txns.head(2), df_txns.tail(2)])

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
83486,2017-08-15,53,932
83487,2017-08-15,54,802


### EDA

#### Check nulls

In [379]:
def show_missing_data(df, df_name):
    """
    Display number and percentage of columns with any missing value
    """
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (
        ((df.isnull().sum() / df.isnull().count()) * 100)
        .sort_values(ascending=False)
        .round(2)
    )
    missing_data = pd.concat([total, percent], axis=1, keys=["# missing", "% missing"])
    
    if missing_data["# missing"].max() > 0:
        print(f'`{df_name}` has null values:')
        print(missing_data[missing_data["# missing"] > 0])
    else:
        print(f'`{df_name}` does not have null values')

In [380]:
show_missing_data(df_train, 'df_train')
show_missing_data(df_test, 'df_test')
show_missing_data(df_txns, 'df_txns')
show_missing_data(df_holidays, 'df_holidays')
show_missing_data(df_oil, 'df_oil')

`df_train` does not have null values
`df_test` does not have null values
`df_txns` does not have null values
`df_holidays` does not have null values
`df_oil` has null values:
            # missing  % missing
dcoilwtico         43       3.53


### Data cleaning & Preprocessing

In [381]:
# df_oil = fill_missing_with_surrounding_mean(df_oil, 'dcoilwtico', 1)
# show_missing_data(df_oil, 'df_oil')

### EDA

In [382]:
# sns.histplot(np.log1p(df_train['sales']), kde=True)

In [383]:
# df_train['sales'].plot(figsize=(10, 6))
# plt.title('Sales over Time')
# plt.show()

In [384]:
# plt.figure(figsize=(5,3))
# sns.lineplot(data=df_oil, x='date', y='dcoilwtico')

### Feature engineering

In [385]:
# def fill_na(df):
#     df["holiday_type"] = df["holiday_type"].fillna("Common")
#     df["locale"] = df["locale"].fillna("Common")
#     df["description"] = df["description"].fillna("Unknown")
#     df["transferred"] = df["transferred"].fillna(False)
#     # TODO: to be replaced by fill_missing_with_surrounding_mean()
#     df["dcoilwtico"] = df["dcoilwtico"].fillna(method="backfill")
#     df["transactions"] = df["transactions"].fillna(0)

#     return df

In [386]:
def create_lag_features_y(df, lags):
    """
    Create column 'sales_t-lag' by taking previous values ​​of column 'sales' based
    on columns 'store_nbr' and 'family'
    """
    for lag in lags:
        df[f"sales_t-{lag}"] = df.groupby(["store_nbr", "family"], observed=True)[
            "sales"
        ].transform(lambda x: x.shift(lag))
    return df

In [387]:
def create_date_features(df):
    # Convert PeriodIndex to DateTimeIndex
    datetime_index = df.index.to_timestamp()

    # Create date features
    df["month"] = datetime_index.month
    df["day_of_month"] = datetime_index.day
    df["day_of_year"] = datetime_index.dayofyear
    df["day_of_week"] = datetime_index.dayofweek
    df["year"] = datetime_index.year
    df["new_year"] = datetime_index.dayofyear == 1

    # Check if it's the 15th of the month
    # df["is_15th"] = datetime_index.day == 15

    # Check if it's the first day of the month
    # df["is_month_start"] = datetime_index.day == 1

    return df

### Model v2

### Pre-processing

In [388]:
df_train['sales'] = np.log1p(df_train['sales'])

In [389]:
# convert field `date` from type datetime64[ns] to period[D]
df_train['date'] = df_train.date.dt.to_period('D')

# remove 'id'ArithmeticError
df_train.drop('id', axis=1, inplace=True)

# create index
df_train = df_train.set_index(["store_nbr", "family", "date"]).sort_index()

df_train.head(2)

sales  onpromotion
store_nbr family     date                             
1         AUTOMOTIVE 2013-01-01  0.000000            0
                     2013-01-02  1.098612            0

### Feature engineering

In [390]:
"""
1) Unstack converts levels of a multi-index into column headers, transforming the
data from a long to a wide format.
-> 'date' becomes the index, and values for 'store_nbr' & 'family' are pivoted into columns
2) if the index is a datetime or period index, you can use .loc[] with a string
that representsa date or a part of a date (like just the year) to filter the data.
"""
y = df_train.unstack(['store_nbr', 'family']).loc["2017"]
y.head(2)

sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-01-01   0.000000       0.0    0.0   0.00000   0.0     0.000000   
2017-01-02   1.791759       0.0    0.0   7.26892   0.0     5.122886   

                                                      ... onpromotion        \
store_nbr                                             ...           9         
family     CELEBRATION  CLEANING     DAIRY      DELI  ...   MAGAZINES MEATS   
date                                                  ...                     
2017-01-01         0.0  0.000000  0.000000  0.000000  ...           0     0   
2017-01-02         0.0  5.808143  5.932245  3.828207  ...           0     0   

                                                                       \
store_nbr                                                               
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY   
date                                                                    
2017-01-01             0            0                       0       0   
2017-01-02            13            0                       0       2   

                                                                      
store_nbr                                                             
family     PREPARED FOODS PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                                  
2017-01-01              0       0                          0       0  
2017-01-02              1       4                          0       0  

[2 rows x 3564 columns]

In [391]:
y.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 227 entries, 2017-01-01 to 2017-08-15
Freq: D
Columns: 3564 entries, ('sales', '1', 'AUTOMOTIVE') to ('onpromotion', '9', 'SEAFOOD')
dtypes: float32(1782), uint32(1782)
memory usage: 3.1 MB


In [392]:
"""
Create seasonal features using DeterministicProcess. In this case, 2 seasonal features:
1) Weekly -> seasonal=True (because date is at day level)
2) Monthly -> CalendarFourier
"""
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

fourier = CalendarFourier(
    freq="M", order=3
)  # 3 sin/cos pairs for "M"onthly seasonality -> sin(1,freq=M), cos(1,freq=M)..

dp = DeterministicProcess(
    index=y.index,
    constant=False,  # dummy feature for bias (y-intercept)
    order=5,  # 5th degree polynomial trend -> creates `trend`, `trend_squared`..
    seasonal=True,  # weekly seasonality (indicators) -> `s(1,7)`, `s(2,7)`..
    additional_terms=[fourier],  # monthly seasonality (fourier)
    drop=True,  # drop terms to avoid collinearity
)

X = dp.in_sample()

In [393]:
X = create_date_features(X)
X.head(3)

,trend,trend_squared,trend_cubed,trend**4,trend**5,"s(1,7)","s(2,7)","s(3,7)","s(4,7)","s(5,7)",...,"sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)",month,day_of_month,day_of_year,day_of_week,year,new_year
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,1.000000,0.000000,1.000000,1,1,1,6,2017,True
2017-01-02,2.0,4.0,8.0,16.0,32.0,0.0,1.0,0.0,0.0,0.0,...,0.394356,0.918958,0.571268,0.820763,1,2,2,0,2017,False
2017-01-03,3.0,9.0,27.0,81.0,243.0,0.0,0.0,1.0,0.0,0.0,...,0.724793,0.688967,0.937752,0.347305,1,3,3,1,2017,False


In [394]:
# @DEV 1: there are missing days (not only the value, but the day itself),
# so first we create all days, and then backfill the missing ones.
# @DEV 2: BUT... if we merge it with train/test, we'll have all days, but we have store & family,
# so probably not the best approach?


# Convert 'date' to DateTimeIndex (assuming 'date' is a column in df_oil)
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil.set_index('date', inplace=True)

# Define your desired date range
start_date = '2013-01-01'
end_date = '2017-08-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Reindex df_oil to include the full date range
df_oil = df_oil.reindex(date_range)

# Fill missing values using backfill
df_oil['dcoilwtico'].bfill(inplace=True)

df_oil.index = df_oil.index.to_period('D')


In [395]:
X = X.merge(df_oil, left_index=True, right_index=True, how='left')

In [396]:
X.head(3)

,trend,trend_squared,trend_cubed,trend**4,trend**5,"s(1,7)","s(2,7)","s(3,7)","s(4,7)","s(5,7)",...,"cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)",month,day_of_month,day_of_year,day_of_week,year,new_year,dcoilwtico
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.000000,0.000000,1.000000,1,1,1,6,2017,True,52.36
2017-01-02,2.0,4.0,8.0,16.0,32.0,0.0,1.0,0.0,0.0,0.0,...,0.918958,0.571268,0.820763,1,2,2,0,2017,False,52.36
2017-01-03,3.0,9.0,27.0,81.0,243.0,0.0,0.0,1.0,0.0,0.0,...,0.688967,0.937752,0.347305,1,3,3,1,2017,False,52.36


In [397]:
# Trying to add lag features

# Define lags (e.g., 1 day, 1 week, 1 month)
lags = [1, 7, 30]

# Create lagged features
df_train_with_lags = create_lag_features_y(df_train.reset_index(), lags)

# Restore the multi-index structure after adding lag features
df_train_with_lags.set_index(["store_nbr", "family", "date"], inplace=True)

# Select the relevant time frame and unstack as done before
y_with_lags = df_train_with_lags.unstack(['store_nbr', 'family']).loc["2017"]

In [398]:
y_with_lags.head(2)

sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-01-01   0.000000       0.0    0.0   0.00000   0.0     0.000000   
2017-01-02   1.791759       0.0    0.0   7.26892   0.0     5.122886   

                                                      ... sales_t-30  \
store_nbr                                             ...          9   
family     CELEBRATION  CLEANING     DAIRY      DELI  ...  MAGAZINES   
date                                                  ...              
2017-01-01         0.0  0.000000  0.000000  0.000000  ...   0.693147   
2017-01-02         0.0  5.808143  5.932245  3.828207  ...   1.098612   

                                                                         \
store_nbr                                                                 
family         MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                      
2017-01-01  6.377059      6.139884     2.079442                1.791759   
2017-01-02  5.799735      6.011267     1.791759                2.197225   

                                                                          \
store_nbr                                                                  
family       POULTRY PREPARED FOODS   PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                       
2017-01-01  6.186807       5.016578  7.445827                   0.693147   
2017-01-02  6.092155       4.543295  7.029241                   0.000000   

                      
store_nbr             
family       SEAFOOD  
date                  
2017-01-01  2.564949  
2017-01-02  2.639057  

[2 rows x 8910 columns]

In [399]:
X.head(2)

,trend,trend_squared,trend_cubed,trend**4,trend**5,"s(1,7)","s(2,7)","s(3,7)","s(4,7)","s(5,7)",...,"cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)",month,day_of_month,day_of_year,day_of_week,year,new_year,dcoilwtico
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.000000,0.000000,1.000000,1,1,1,6,2017,True,52.36
2017-01-02,2.0,4.0,8.0,16.0,32.0,0.0,1.0,0.0,0.0,0.0,...,0.918958,0.571268,0.820763,1,2,2,0,2017,False,52.36


In [400]:
lag_intervals = [7, 30]  # Example lags: 1 week, 1 month

for lag in lag_intervals:
    lagged_feature_name = f'dcoilwtico_lag_{lag}'
    X[lagged_feature_name] = X['dcoilwtico'].shift(lag)

# Backfill NaN values in the newly created lagged feature columns
for lag in lag_intervals:
    lagged_feature_name = f'dcoilwtico_lag_{lag}'
    X[lagged_feature_name].bfill(inplace=True)

### Split Train/Test

In [401]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_with_lags, test_size=0.1, random_state=RANDOM_STATE
)

In [402]:
from sklearn.preprocessing import RobustScaler

transformerL = RobustScaler().fit(X_train)

X_test = transformerL.transform(X_test)
X_train = transformerL.transform(X_train)

In [403]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_models(regressors, X, y):
    results = []
    for regressor in regressors:
        y_pred = regressor.predict(X)
        # TODO: shouldn't happen, but...
        y_pred[y_pred < 0] = 0
        mae = mean_absolute_error(y, y_pred)
        rmse = np.sqrt(mean_squared_error(y, y_pred))
        results.append(
            {
                "Model": type(regressor).__name__,
                "MAE": mae,
                "RMSE": rmse,
            }
        )
    sorted_results = sorted(results, key=lambda x: x["RMSE"], reverse=False)
    print(
        f"{'Model':<25} {'RMSE':<11} {'MAE':<8}"
    )
    print("-" * 72)
    for result in sorted_results:
        metrics = f"{result['RMSE']:<11.5f} {result['MAE']:<8.2f}"
        print(f"{result['Model']:<25} {metrics}")

In [404]:
df_train.head(3)
# df_train[df_train.index.get_level_values('date')> '2016-10-01']['sales'].groupby('date').sum().head(40)

sales  onpromotion
store_nbr family     date                             
1         AUTOMOTIVE 2013-01-01  0.000000            0
                     2013-01-02  1.098612            0
                     2013-01-03  1.386294            0

### Lasso & Ridge

In [405]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn import metrics

lasso = Lasso(alpha=1, fit_intercept=True, max_iter=7000).fit(X_train, y_train)

ridge = Ridge(alpha=0.4, fit_intercept=True, max_iter=7000).fit(X_train, y_train)


/Users/wint3rmute/Gro/datascilabs/vtensorflow/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


In [406]:
models = [ridge, lasso]
evaluate_models(models, X_test, y_test)

Model                     RMSE        MAE     
------------------------------------------------------------------------
Ridge                     1.91459     0.58    
Lasso                     2.21905     0.73    


with lagged oil in 'X':

Ridge                     1.91459     0.58    
Lasso                     2.21905     0.73    

with lagged features 1,7 & 30 on 'y':

Ridge                     1.99389     0.60    
Lasso                     2.32290     0.74 

before lagged:

Ridge                     3.10469     1.04    
Lasso                     3.61448     1.32   

In [407]:
"""
Based on your description, df_test has 28,512 records, spanning dates from 2017-08-16 to 2017-08-31,
with approximately 1,782 records per day. This structure suggests that for each day, there are 1,782
combinations of 'store_nbr' and 'family' products. Given that there are 16 days in your test date range,
this aligns with the total record count (16 days * 1,782 records/day = 28,512 records).
"""

"\nBased on your description, df_test has 28,512 records, spanning dates from 2017-08-16 to 2017-08-31,\nwith approximately 1,782 records per day. This structure suggests that for each day, there are 1,782\ncombinations of 'store_nbr' and 'family' products. Given that there are 16 days in your test date range,\nthis aligns with the total record count (16 days * 1,782 records/day = 28,512 records).\n"

In [408]:
import copy

df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

X_test = dp.out_of_sample(steps=16)
X_test.index.name = 'date'
X_test['NewYear'] = (X_test.index.dayofyear == 1)

X_test_copy = copy.deepcopy(X_test)

X_test = transformerL.transform(X_test)

predictionL = lasso.predict(X_test)
predictionL[predictionL < 0] = 0

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- NewYear
Feature names seen at fit time, yet now missing:
- day_of_month
- day_of_week
- day_of_year
- dcoilwtico
- dcoilwtico_lag_30
- ...


In [ ]:
y_submit = pd.DataFrame(np.expm1(predictionL), index=X_test_copy.index, columns=y.columns)
# y_submit = pd.DataFrame(predictionL, index=X_test_copy.index, columns=y.columns)
y_submit = y_submit.stack(['store_nbr', 'family'])
y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])

NameError: name 'predictionL' is not defined

In [ ]:
y_submit

id        sales
date       store_nbr family                                          
2017-08-16 1         AUTOMOTIVE                  3000888     3.283707
                     BABY CARE                   3000889     0.000000
                     BEAUTY                      3000890     2.689846
                     BEVERAGES                   3000891  2004.868932
                     BOOKS                       3000892     0.320290
...                                                  ...          ...
2017-08-31 9         POULTRY                     3029395   418.607620
                     PREPARED FOODS              3029396   117.379339
                     PRODUCE                     3029397  1579.561665
                     SCHOOL AND OFFICE SUPPLIES  3029398     3.194469
                     SEAFOOD                     3029399    16.123541

[28512 rows x 2 columns]

In [ ]:
# y_submit.to_csv('submission.csv', index=False)

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=13)

# from sklearn.preprocessing import RobustScaler

# transformerKERAS = RobustScaler().fit(X_train)

# X_val = transformerKERAS.transform(X_val)
# X_train = transformerKERAS.transform(X_train)

# from tensorflow.keras.callbacks import ReduceLROnPlateau
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=0.000001, verbose=1, mode='min')

# def create_model():

#     model = Sequential()
#     model.add(Dense(units=500, activation='relu', input_dim=19))
#     model.add(Dense(units=2000, activation='relu'))
#     model.add(Dense(units=1500, activation='relu'))
#     model.add(Dense(units=1782, activation='swish'))

#     model.compile(loss='mae', optimizer='adam')
    
#     return model

# model = create_model()
# model.fit(X_train, y_train, epochs=5000, batch_size=2000, validation_data=(X_val, y_val),callbacks=[reduce_lr])

# y_pred1 = model.predict(X_val)
# #print(model1.score(X_val, y_val))

# y_pred1[y_pred1 < 0] = 0

# print('Mean Absolute Error:', metrics.mean_absolute_error(y_val, y_pred1))
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val, y_pred1)))

# print('======')